In [ ]:
# --- 1. INSTALL LIBRARIES ---
!pip install -q gradio ultralytics opencv-python numpy

# --- 2. IMPORT LIBRARIES ---
import gradio as gr
import cv2
import os
import random
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# --- 3. LOAD YOUR AI MODEL ---
# Make sure your Google Drive is mounted and the path is correct
from google.colab import drive
drive.mount('/content/drive')
MODEL_PATH = '/content/best (1).pt'

try:
    model = YOLO(MODEL_PATH)
    print("✅ AI Model loaded successfully.")
except Exception as e:
    print(f"❌ CRITICAL ERROR: Could not load the model. Please check the path: {e}")
    raise SystemExit("Model loading failed.")

# --- 4. COMPONENT DATABASE WITH ALL DETAILS ---
COMPONENT_DATABASE = {
    'ic': {'tier': 'High-Value', 'score': 10, 'description': "are prime candidates for harvesting due to their high resale value and functionality.", 'hazardous': False},
    'coil': {'tier': 'Medium-Value', 'score': 3, 'description': "are valuable for their copper content and are often reused in power circuits.", 'hazardous': False},
    'mosfet': {'tier': 'Medium-Value', 'score': 3, 'description': "are key components in power regulation and are frequently harvested for repair.", 'hazardous': True},
    'transistor': {'tier': 'Low-Value', 'score': 0, 'description': "", 'hazardous': True},
    'default': {'tier': 'Low-Value', 'score': 0, 'description': "", 'hazardous': False}
}

# --- 5. THE MAIN ANALYSIS FUNCTION ---
def analyze_pcb_with_gradio(input_image):
    img = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    results = model(img)

    # --- Analysis Logic ---
    component_counts = defaultdict(int)
    reusability_score = 0
    total_components = 0
    detailed_factors = defaultdict(list)
    is_hazardous = False

    for box in results[0].boxes:
        total_components += 1
        class_name = model.names[int(box.cls[0])]
        confidence = float(box.conf[0])
        component_counts[class_name] += 1

        data = COMPONENT_DATABASE.get(class_name, COMPONENT_DATABASE['default'])
        reusability_score += data['score']

        if data['score'] > 0:
            detailed_factors[class_name.capitalize()].append(data['description'])

        if data['hazardous']:
            is_hazardous = True

        # Drawing bounding boxes
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = f"{class_name} {confidence:.2f}"
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    if total_components > 50:
        reusability_score += 15
        detailed_factors["Board Complexity"].append("This board has a high component density, often indicating a more valuable board.")

    # --- 6. SEGREGATION RECOMMENDATION LOGIC ---
    if is_hazardous:
        recommendation = "☣️ HAZARDOUS: Segregate for special disposal."
    elif reusability_score >= 20:
        recommendation = "♻️ REUSABLE: Segregate for component harvesting."
    else:
        recommendation = "⚙️ GENERAL RECYCLING: Route to standard shredding process."

    # --- 7. FORMAT THE DETAILED REPORT ---
    report = "--- FULL BOARD ANALYSIS REPORT ---\n\n"
    report += "[ 1. COMPONENT SUMMARY ]\n"
    for component, count in component_counts.items():
        report += f"- {component.capitalize()}: {count} units\n"

    report += "\n[ 2. DETAILED REUSABILITY ANALYSIS ]\n"
    if reusability_score >= 40: verdict = "Prime Candidate for Component Harvesting."
    elif reusability_score >= 10: verdict = "Good Potential for Reuse."
    else: verdict = "Low Potential for Reuse."
    report += f"Reusability Score: {reusability_score} / 100\n"
    report += f"Verdict: {verdict}\n"

    if detailed_factors:
        report += "\nFactors Considered in This Decision:\n"
        for component_name, descriptions in detailed_factors.items():
            unique_description = descriptions[0]
            count = component_counts.get(component_name.lower(), len(descriptions))
            if "Board Complexity" in component_name:
                 report += f"  {component_name}: {unique_description}\n"
            else:
                 report += f"  {count} x {component_name}: Detected. These {unique_description}\n"

    annotated_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return recommendation, report, annotated_image

# --- 8. CREATE AND LAUNCH THE GRADIO INTERFACE ---
iface = gr.Interface(
    fn=analyze_pcb_with_gradio,
    inputs=gr.Image(type="numpy", label="Upload PCB Image"),
    outputs=[
        # ✨ MODIFIED: Recommendation box is now a single line again
        gr.Textbox(label="✅ Final Segregation Recommendation"),
        # ✨ MODIFIED: Detailed report box is large
        gr.Textbox(label="Detailed Analysis Report", lines=20),
        gr.Image(type="numpy", label="Detection Result", height=600)
    ],
    title="E-Waste PCB Analyzer & Sorter",
    description="Upload a circuit board image to get a detailed analysis and a clear segregation recommendation.",
    allow_flagging="never"
)

# Launch the app with a public link
iface.launch(share=True, debug=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ AI Model loaded successfully.


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f6cc9ac71127ca243f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6iTMOIcJZY6uftvqd3iI")
project = rf.workspace("srmhack").project("pcb-components-r8l8r-pm1ig")
version = project.version(1)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to PCB-components-1 in yolov8:: 100%|██████████| 134/134 [00:00<00:00, 935.33it/s] 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Task
Set up a web application using a Python framework (Flask or Django) to host a YOLO model for object detection. The application should have a web interface for image uploads, process the images using the YOLO model, and display the detection results (bounding boxes and component summary) to the user. Finally, provide instructions on how to deploy and access the application.

## Set up a web framework

### Subtask:
Choose a Python web framework like Flask or Django to build your web application.


**Reasoning**:
Flask is a good choice for this project as it is a microframework and the application is relatively simple, focusing on image upload and model inference. The first step is to install Flask using pip.



## Create a web interface

### Subtask:
Design a simple HTML interface with a file upload form for users to submit images.


**Reasoning**:
Create the HTML file with a file upload form as instructed.



## Implement backend logic

### Subtask:
Write Python code to receive the uploaded image, save it temporarily, and pass it to your trained YOLO model for inference.


**Reasoning**:
Write the Python code for the Flask application to handle image uploads, perform inference using the YOLO model, and prepare the results.



In [ ]:
!pip install ultralytics

**Reasoning**:
The previous code block failed because the `ultralytics` library was not found. I need to install it before importing and using the YOLO model.



In [ ]:
!pip install ultralytics

**Reasoning**:
Now that `ultralytics` is installed, I can re-run the code to define the Flask application, load the YOLO model, and handle the image upload and inference.



## Implement backend logic

### Subtask:
Retry implementing the backend logic to receive the uploaded image, save it temporarily, and pass it to your trained YOLO model for inference.


**Reasoning**:
Update the model path in the code to the correct location and attempt to load the model again.



## Display results

### Subtask:
Render the results back to the user, potentially by drawing bounding boxes on the uploaded image and displaying the component summary.


**Reasoning**:
The previous attempts to load the model from Google Drive failed due to mounting issues. I will modify the Flask application code to render an HTML template that displays the detected image and component summary after processing, create the necessary HTML template file, add a route to serve the processed image, and ensure the Flask app is set up to run, incorporating necessary imports for rendering templates and serving static files. I will also adjust the model loading to use the path where the model was successfully downloaded in a previous step of the notebook, which is `/content/runs/detect/train/weights/best.pt`, to avoid the Google Drive mounting dependency.



**Reasoning**:
The previous code block failed because it attempted to use the `%%writefile` magic command within a Python script block. I need to separate the creation of the `results.html` file using `%%writefile` into its own cell and then define the Flask application code in a separate cell. I will also create the `templates` directory before attempting to write the HTML file into it.



**Reasoning**:
Now that the `templates` directory is created, I can write the `results.html` content into the file using the `%%writefile` magic command in a separate cell.



**Reasoning**:
With the `results.html` template successfully created, I can now define the Flask application code that utilizes this template and includes the routes for serving the processed images.



## Deploy the application

### Subtask:
Choose a hosting platform (e.g., Heroku, Google App Engine, AWS) to deploy your web application so it's accessible as a website.


**Reasoning**:
The previous steps have set up the Flask application and the necessary templates. The current subtask is to select a hosting platform and prepare the application for deployment, which involves creating a `requirements.txt` file listing all necessary Python packages. This file is crucial for the hosting platform to set up the correct environment.



In [ ]:
%%writefile requirements.txt
Flask
ultralytics
opencv-python
numpy
Pillow # Added Pillow as it's often a dependency for image processing and ultralytics

## Summary:

### Data Analysis Key Findings

*   Flask was chosen as the web framework for the application.
*   An `index.html` file was created with a form for image uploads.
*   A `results.html` template was created to display the processed image and component summary using Jinja2 templating.
*   The Flask application code was defined with routes for the home page (`/`), file upload and processing (`/upload`), and serving processed images (`/processed_images/<filename>`).
*   The `/upload` route includes logic to save the uploaded image, perform inference using a YOLO model, draw bounding boxes and labels on the image, count components, save the processed image, and render the `results.html` template.
*   A `requirements.txt` file was created listing the necessary Python dependencies (`Flask`, `ultralytics`, `opencv-python`, `numpy`, `Pillow`) for deployment.
*   The process encountered and reported errors related to loading the YOLO model, initially due to `ultralytics` not being installed, then a `FileNotFoundError` for the model path, and subsequently a `ValueError` during Google Drive mounting attempts. Despite these errors preventing full application functionality, the code for handling uploads, inference processing, and rendering results was implemented.

### Insights or Next Steps

*   The primary next step is to resolve the YOLO model loading issue by ensuring the model file is accessible at the specified `model_path` within the environment where the Flask application is run or deployed. This might involve adjusting the path, ensuring the file exists, or correctly configuring access to cloud storage if the model is stored there.
*   To make the application fully functional and accessible via a web browser, the Flask application needs to be executed using `app.run()` (for local testing) or deployed to a hosting platform (like Google Cloud Platform, Heroku, etc.) using a production-ready server (like Gunicorn or uWSGI). The deployment process will also require configuring the hosting environment to serve the application and handle file uploads and static assets (like the processed images).
